In [8]:
using LinearAlgebra, Plots, DelimitedFiles
include("hamming.jl")
include("chase.jl")
import .hamming.create_GH
import .chase.SISO_Pyndiah

In [9]:
G, H = create_GH(6)
k, n = size(G)
batch_size = 100
min_num = 30
iter_num = 6
window_size = 7
Α = [0.0, 0.2, 0.3, 0.5, 0.7, 0.9, 1.0, 1.0]

8-element Vector{Float64}:
 0.0
 0.2
 0.3
 0.5
 0.7
 0.9
 1.0
 1.0

In [10]:
function turbo(Y)
    W = zeros(Float64, n, n)
    R = Y + Α[1] * W
    for i = 1:iter_num
        # R = R + Α[i] * W
        if (isodd(i))
            for j = 1:n
                W[j, :] = SISO_Pyndiah(H, R[j, :], i) - Y[j, :]
            end
        end
        if (iseven(i))
            for j = 1:n
                W[:, j] = SISO_Pyndiah(H, R[:, j], i) - Y[:, j]
            end
        end
        R = Y + Α[i+1] * W
    end
    D = ifelse.(R .> 0, 0, 1)
    return D
end

turbo (generic function with 1 method)

In [11]:
function one_iter(σ, algorithm)
    m = rand((0, 1), k, k)
    x = mod.(G' * m, 2)
    x = mod.(x * G, 2)
    y = (-1) .^ x + randn(size(x)) * σ
    z = algorithm(y)
    return sum(mod.(x + z, 2))

end
one_iter(1, turbo)

732

In [12]:
function error_rate(σ, algorithm)
    cnt = 0
    tmp = 0
    batch_num = 0
    while (cnt < min_num)
        tot = 0
        for i = 1:batch_size
            ttt = one_iter(σ, algorithm)
            if (ttt == 0)
                continue
            end
            tot += 1
            tmp += ttt
        end
        cnt += tot
        batch_num += 1
    end
    p = tmp / (batch_num * batch_size * n * n)
    return p
end

error_rate (generic function with 1 method)

In [16]:
iter_num = 6
error_rate(10^-0.3, turbo)

6.13138555936073e-7